In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 5237bfbc-06d7-4a5a-b56e-ff1002bb1751, 3, Finished, Available)

In [2]:
%%sql
create table if not EXISTS TMDW_FinalProj.gld_fact_sale
(
OID	string	,
Price float,
Quantity	float	,
Sales	float	,
Discount	float	,
Profit	float	,
Ship_Cost	float	,
Order_Date	date	,
Ship_Date	date	,
PID	long	,
OrderPriority_ID	Long	,
ShipMode_ID	Long	,
Cust_ID	string	,
Order_Year	integer	,
Order_Month	integer	,
Created_TS	timestamp	,
Modified_TS	timestamp	
)
using DELTA
PARTITIONED by (Order_Year,Order_Month)

StatementMeta(, 5237bfbc-06d7-4a5a-b56e-ff1002bb1751, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Maximum_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from TMDW_FinalProj.gld_fact_sale").first()[0]

StatementMeta(, 5237bfbc-06d7-4a5a-b56e-ff1002bb1751, 5, Finished, Available)

In [4]:
Maximum_Date

StatementMeta(, 5237bfbc-06d7-4a5a-b56e-ff1002bb1751, 6, Finished, Available)

'1900-01-01'

In [10]:
dataframe_bze= spark.sql("""
    SELECT 
        bronzesales.OID,
        bronzesales.Sales AS Price,
        bronzesales.Quantity,
        bronzesales.Sales * bronzesales.Quantity AS Sales,
        bronzesales.Discount,
        bronzesales.Profit,
        bronzesales.Ship_Cost,
        bronzesales.Order_Date,
        bronzesales.Ship_Date,
        gld_product.PID,
        gld_orderpriority.OP_ID,
        gld_shipmode.SM_ID,
        bronzesales.Cust_ID,
        Year(bronzesales.Order_Date) AS Order_Year,
        Month(bronzesales.Order_Date) AS Order_Month
    FROM 
        TMDW_FinalProj.bronzesales 
    INNER JOIN 
        TMDW_FinalProj.gld_product 
    ON 
        bronzesales.Product = gld_product.Product 
        AND bronzesales.Product_Category = gld_product.Product_Category
    INNER JOIN 
        TMDW_FinalProj.gld_shipmode 
    ON 
        bronzesales.Ship_Mode = gld_shipmode.Ship_Mode
    INNER JOIN 
        TMDW_FinalProj.gld_orderpriority 
    ON 
        bronzesales.Order_Priority = gld_orderpriority.Order_Priority
""")


StatementMeta(, 5237bfbc-06d7-4a5a-b56e-ff1002bb1751, 12, Finished, Available)

In [11]:
dataframe_bze.createOrReplaceTempView("View_FactSale")

StatementMeta(, 5237bfbc-06d7-4a5a-b56e-ff1002bb1751, 13, Finished, Available)

In [14]:
%%sql
merge into TMDW_FinalProj.gld_fact_sale as gfs
using View_FactSale as vfs 
on gfs.Order_Year=vfs.Order_Year and gfs.Order_Month=vfs.Order_Month and gfs.OID=vfs.OID
when matched then 
update SET
gfs.Sales	=	vfs.Sales	,
gfs.Price	=	vfs.Price	,
gfs.Quantity	=	vfs.Quantity	,
gfs.Discount	=	vfs.Discount	,
gfs.Profit	=	vfs.Profit	,
gfs.Ship_Cost	=	vfs.Ship_Cost	,
gfs.Order_Date	=	vfs.Order_Date	,
gfs.Ship_Date	=	vfs.Ship_Date	,
gfs.PID	=	vfs.PID	,
gfs.OrderPriority_ID	=	vfs.OP_ID	,
gfs.ShipMode_ID	=	vfs.SM_ID	,
gfs.Cust_ID	=	vfs.Cust_ID	,
gfs.Modified_TS	=	current_timestamp()	

when not matched then
INSERT
(
gfs.OID	,
gfs.Sales	,
gfs.Price,
gfs.Quantity	,
gfs.Discount	,
gfs.Profit	,
gfs.Ship_Cost	,
gfs.Order_Date	,
gfs.Ship_Date	,
gfs.PID	,
gfs.OrderPriority_ID	,
gfs.ShipMode_ID	,
gfs.Cust_ID	,
gfs.Order_Year	,
gfs.Order_Month	,
gfs.Created_TS	,
gfs.Modified_TS	

)
VALUES
(
vfs.OID	,
vfs.Sales	,
vfs.Price,
vfs.Quantity	,
vfs.Discount	,
vfs.Profit	,
vfs.Ship_Cost	,
vfs.Order_Date	,
vfs.Ship_Date	,
vfs.PID	,
vfs.OP_ID	,
vfs.SM_ID	,
vfs.Cust_ID	,
vfs.Order_Year	,
vfs.Order_Month	,
current_timestamp()	,
current_timestamp()

)

StatementMeta(, 5237bfbc-06d7-4a5a-b56e-ff1002bb1751, 16, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>